In [1]:
"""
Nepal District Vulnerability Analysis - Feature Engineering
Step 3: Create features and target variable for ML analysis
Improved Version: Fixed data leakage issues and enhanced feature engineering
"""

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Setup paths
BASE_PATH = Path(r'C:\Users\saurav\Downloads\SEVI_Nepal_Project')
PROCESSED_PATH = BASE_PATH / 'data' / 'processed'
FINAL_PATH = BASE_PATH / 'data' / 'final'
VIS_PATH = BASE_PATH / 'results' / 'figures'

# Create directories
for path in [FINAL_PATH, VIS_PATH]:
    path.mkdir(parents=True, exist_ok=True)

print("="*80)
print("FEATURE ENGINEERING FOR VULNERABILITY ANALYSIS - IMPROVED VERSION")
print("="*80)

# ============================================================================
# STEP 1: Load cleaned data
# ============================================================================

print("\n1. Loading data...")
df = pd.read_csv(PROCESSED_PATH / 'simple_merged_districts.csv')
print(f"   ✓ Loaded {len(df)} districts with {len(df.columns)} columns")
print(f"   District columns: {df[['ID', 'DISTRICT_NAME']].head(3).to_string(index=False)}")

# Check for province data
if 'PROVINCE' in df.columns:
    print(f"   ✓ Province data available: {df['PROVINCE'].nunique()} provinces")
    print(f"   Province distribution:\n{df['PROVINCE'].value_counts().sort_index()}")

# ============================================================================
# STEP 2: Identify feature groups
# ============================================================================

print("\n2. Identifying feature groups...")

# Group features by type
feature_groups = {
    'Walls': [col for col in df.columns if col.startswith('WAL_')],
    'Water': [col for col in df.columns if col.startswith('DRI_')],
    'Toilets': [col for col in df.columns if col.startswith('TOI_')]
}

for group_name, features in feature_groups.items():
    if features:
        print(f"   {group_name}: {len(features)} features")
        print(f"     Samples: {', '.join(features[:3])}...")

# ============================================================================
# STEP 3: Convert counts to percentages
# ============================================================================

print("\n3. Converting counts to percentages...")

def safe_percentage(df, count_features):
    """Safely convert counts to percentages"""
    total = df[count_features].sum(axis=1)
    # Replace zeros with NaN to avoid division by zero
    total = total.replace(0, np.nan)
    
    percentages = df[count_features].div(total, axis=0) * 100
    # Fill NaN values with 0 (for districts with no data in a category)
    percentages = percentages.fillna(0)
    
    # Rename columns
    percentages.columns = [f'PCT_{col}' for col in count_features]
    
    return percentages

# Create percentage features
pct_df = pd.DataFrame()
for group_name, features in feature_groups.items():
    if features:  # Check if list is not empty
        group_pct = safe_percentage(df, features)
        pct_df = pd.concat([pct_df, group_pct], axis=1)
        print(f"   ✓ Created {len(features)} percentage features for {group_name}")

# Combine with original data
df_pct = pd.concat([df[['ID', 'DISTRICT_NAME']], pct_df], axis=1)
print(f"   Total percentage features: {len(pct_df.columns)}")

# ============================================================================
# STEP 4: Create vulnerability indices (for target creation ONLY)
# ============================================================================

print("\n4. Creating vulnerability indices (for target only)...")

# Define vulnerable materials/sources - USED ONLY FOR TARGET CREATION
vuln_mapping = {
    # Housing vulnerability (poor quality materials)
    'HOUSING_VULN': [
        'PCT_WAL_MUD_BONDED_BRICKS_STONE',
        'PCT_WAL_BAMBOO',
        'PCT_WAL_UNBAKED_BRICKS',
        'PCT_WAL_GALVANIZED_SHEET',
        'PCT_WAL_OTHER'
    ],
    
    # Water vulnerability (unsafe sources)
    'WATER_VULN': [
        'PCT_DRI_UNCOVERED_WELL_KUWA',
        'PCT_DRI_SPOUT_WATER',
        'PCT_DRI_RIVER_STREAM',
        'PCT_DRI_OTHER_SOURCES'
    ],
    
    # Sanitation vulnerability (poor facilities)
    'SANITATION_VULN': [
        'PCT_TOI_PIT_TOILET',
        'PCT_TOI_WITHOUT_TOILET_FACILITY'
    ]
}

# Calculate indices for TARGET ONLY
vuln_indices_target = pd.DataFrame()
for index_name, features in vuln_mapping.items():
    # Use only features that exist in our data
    existing_features = [f for f in features if f in df_pct.columns]
    if existing_features:
        vuln_indices_target[index_name] = df_pct[existing_features].mean(axis=1)
        print(f"   ✓ {index_name}: average of {len(existing_features)} features (TARGET ONLY)")

# ============================================================================
# STEP 5: Create additional engineered features (for ML features)
# ============================================================================

print("\n5. Creating additional engineered features for ML...")

# Create ratio-based features that are informative but not directly used in target
engineered_features = {}

# 1. Infrastructure quality ratios
# Safe vs unsafe walls ratio
if 'PCT_WAL_CEMENT_BONDED_BRICKS_STONE' in df_pct.columns:
    safe_walls = df_pct['PCT_WAL_CEMENT_BONDED_BRICKS_STONE'].fillna(0)
    unsafe_walls = df_pct.get('PCT_WAL_MUD_BONDED_BRICKS_STONE', 0).fillna(0) + \
                   df_pct.get('PCT_WAL_UNBAKED_BRICKS', 0).fillna(0)
    engineered_features['WALL_QUALITY_RATIO'] = (safe_walls + 1) / (unsafe_walls + 1)  # +1 to avoid division by zero

# 2. Water access quality
safe_water_features = []
unsafe_water_features = []

for feat in df_pct.columns:
    if 'PCT_DRI_' in feat:
        if any(term in feat for term in ['TAP_PIPED', 'TUBEWELL', 'COVERED_WELL']):
            safe_water_features.append(feat)
        elif any(term in feat for term in ['UNCOVERED', 'RIVER', 'OTHER']):
            unsafe_water_features.append(feat)

safe_water = df_pct[safe_water_features].sum(axis=1).fillna(0) if safe_water_features else 0
unsafe_water = df_pct[unsafe_water_features].sum(axis=1).fillna(0) if unsafe_water_features else 0
engineered_features['WATER_ACCESS_RATIO'] = (safe_water + 1) / (unsafe_water + 1)

# 3. Sanitation access
if 'PCT_TOI_FLUSH_PUBLIC_SEWERAGE' in df_pct.columns:
    good_sanitation = df_pct['PCT_TOI_FLUSH_PUBLIC_SEWERAGE'].fillna(0) + \
                      df_pct.get('PCT_TOI_FLUSH_SEPTIC_TANK', 0).fillna(0)
    poor_sanitation = df_pct.get('PCT_TOI_WITHOUT_TOILET_FACILITY', 0).fillna(0)
    engineered_features['SANITATION_RATIO'] = (good_sanitation + 1) / (poor_sanitation + 1)

# 4. Diversity indices (Simpson's Diversity Index)
def simpson_diversity(series):
    """Calculate Simpson's Diversity Index for percentage data"""
    proportions = series / 100
    return 1 - (proportions ** 2).sum()

# Wall material diversity
wall_features = [col for col in df_pct.columns if 'PCT_WAL_' in col]
if wall_features:
    engineered_features['WALL_DIVERSITY'] = df_pct[wall_features].apply(simpson_diversity, axis=1)

# Water source diversity
water_features = [col for col in df_pct.columns if 'PCT_DRI_' in col]
if water_features:
    engineered_features['WATER_SOURCE_DIVERSITY'] = df_pct[water_features].apply(simpson_diversity, axis=1)

# Convert to DataFrame
engineered_df = pd.DataFrame(engineered_features)
print(f"   ✓ Created {len(engineered_features)} engineered features for ML")

# Combine all features
df_engineered = pd.concat([df_pct, engineered_df], axis=1)

# Add province information if available
if 'PROVINCE' in df.columns:
    df_engineered['PROVINCE'] = df['PROVINCE']
    print(f"   ✓ Added province information")

# Add vulnerability indices (FOR TARGET ONLY - will be separated later)
df_engineered = pd.concat([df_engineered, vuln_indices_target], axis=1)

# ============================================================================
# STEP 6: Create overall vulnerability score (TARGET)
# ============================================================================

print("\n6. Creating overall vulnerability score (TARGET)...")

# Weighted average of vulnerability components (this is our target)
weights = {
    'HOUSING_VULN': 0.4,      # Housing is often most important
    'WATER_VULN': 0.3,        # Water access is critical
    'SANITATION_VULN': 0.3     # Sanitation affects health
}

# Calculate weighted score (TARGET VARIABLE)
vulnerability_components = []
for index, weight in weights.items():
    if index in df_engineered.columns:
        vulnerability_components.append(df_engineered[index] * weight)

if vulnerability_components:
    df_engineered['VULNERABILITY_SCORE'] = sum(vulnerability_components)
    
    # Normalize to 0-100 scale for easier interpretation
    min_score = df_engineered['VULNERABILITY_SCORE'].min()
    max_score = df_engineered['VULNERABILITY_SCORE'].max()
    if max_score > min_score:  # Avoid division by zero
        df_engineered['VULNERABILITY_SCORE_NORM'] = (
            (df_engineered['VULNERABILITY_SCORE'] - min_score) / 
            (max_score - min_score) * 100
        )
    else:
        df_engineered['VULNERABILITY_SCORE_NORM'] = 50  # Default mid-point
        
    print(f"   ✓ Created vulnerability score (range: {min_score:.1f} - {max_score:.1f})")
    print(f"   ✓ Normalized to 0-100 scale")

# ============================================================================
# STEP 7: Create target categories (TARGET)
# ============================================================================

print("\n7. Creating vulnerability categories (TARGET)...")

if 'VULNERABILITY_SCORE_NORM' in df_engineered.columns:
    # Create 4 categories using quartiles
    df_engineered['VULNERABILITY_CATEGORY'] = pd.qcut(
        df_engineered['VULNERABILITY_SCORE_NORM'],
        q=4,
        labels=['Low', 'Medium', 'High', 'Very High']
    )
    
    # Show distribution
    print("   Category distribution:")
    category_counts = df_engineered['VULNERABILITY_CATEGORY'].value_counts().sort_index()
    for category, count in category_counts.items():
        percentage = (count / len(df_engineered)) * 100
        print(f"     {category}: {count} districts ({percentage:.1f}%)")
    
    # Show extremes
    print("\n   Most vulnerable districts:")
    top_3 = df_engineered.nlargest(3, 'VULNERABILITY_SCORE_NORM')[['DISTRICT_NAME', 'VULNERABILITY_SCORE_NORM', 'VULNERABILITY_CATEGORY']].copy()
    top_3['VULNERABILITY_SCORE_NORM'] = top_3['VULNERABILITY_SCORE_NORM'].round(1)
    print(top_3.to_string(index=False))
    
    print("\n   Least vulnerable districts:")
    bottom_3 = df_engineered.nsmallest(3, 'VULNERABILITY_SCORE_NORM')[['DISTRICT_NAME', 'VULNERABILITY_SCORE_NORM', 'VULNERABILITY_CATEGORY']].copy()
    bottom_3['VULNERABILITY_SCORE_NORM'] = bottom_3['VULNERABILITY_SCORE_NORM'].round(1)
    print(bottom_3.to_string(index=False))

# ============================================================================
# STEP 8: Prepare ML dataset (SEPARATE FEATURES FROM TARGET)
# ============================================================================

print("\n8. Preparing ML dataset (separating features from target)...")

# IDENTIFY FEATURE TYPES:

# 1. Raw percentage features (PCT_* but NOT _VULN indices)
raw_pct_features = [col for col in df_engineered.columns 
                    if col.startswith('PCT_') and not col.endswith('_VULN')]

print(f"   Raw percentage features: {len(raw_pct_features)}")

# 2. Engineered features (ratios, diversity indices)
engineered_ml_features = [col for col in df_engineered.columns 
                          if col in engineered_features.keys()]

print(f"   Engineered ML features: {len(engineered_ml_features)}")

# 3. Categorical features (for one-hot encoding later)
categorical_features = []
if 'PROVINCE' in df_engineered.columns:
    categorical_features = ['PROVINCE']
    print(f"   Categorical features: {len(categorical_features)}")

# 4. VULNERABILITY INDICES - THESE ARE TARGET-RELATED, NOT FEATURES!
# DO NOT INCLUDE THESE IN ML FEATURES TO AVOID DATA LEAKAGE
vuln_indices = [col for col in df_engineered.columns if col.endswith('_VULN')]
print(f"   Vulnerability indices (EXCLUDED from features to avoid data leakage): {len(vuln_indices)}")

# Combine ALL features (excluding target-related indices)
all_ml_features = raw_pct_features + engineered_ml_features + categorical_features
print(f"   Total ML features: {len(all_ml_features)}")

# Create ML dataset
ml_df = df_engineered[['ID', 'DISTRICT_NAME'] + all_ml_features].copy()

# Add target variables
if 'VULNERABILITY_CATEGORY' in df_engineered.columns:
    ml_df['TARGET_CATEGORY'] = df_engineered['VULNERABILITY_CATEGORY']
    ml_df['TARGET_SCORE'] = df_engineered['VULNERABILITY_SCORE_NORM']
    print(f"   ✓ Added target variables: TARGET_CATEGORY and TARGET_SCORE")

# Display feature information
print(f"\n   Feature groups summary:")
print(f"     - Raw percentages: {len(raw_pct_features)} features")
print(f"     - Engineered ratios: {len(engineered_ml_features)} features")
print(f"     - Categorical: {len(categorical_features)} features")
print(f"     - TOTAL for ML: {len(all_ml_features)} features")

print(f"\n   Sample features from each group:")
print(f"     Raw PCT features: {', '.join(raw_pct_features[:3])}...")
if engineered_ml_features:
    print(f"     Engineered features: {', '.join(engineered_ml_features)}")
if categorical_features:
    print(f"     Categorical features: {', '.join(categorical_features)}")

# ============================================================================
# STEP 9: Save outputs
# ============================================================================

print("\n9. Saving outputs...")

# Save full engineered dataset
df_engineered.to_csv(FINAL_PATH / 'nepal_districts_engineered.csv', index=False)
print(f"   ✓ Full dataset: {FINAL_PATH / 'nepal_districts_engineered.csv'}")

# Save ML-ready dataset
ml_df.to_csv(FINAL_PATH / 'nepal_districts_ml_ready.csv', index=False)
print(f"   ✓ ML dataset: {FINAL_PATH / 'nepal_districts_ml_ready.csv'}")

# Save detailed feature documentation
with open(FINAL_PATH / 'feature_documentation.txt', 'w') as f:
    f.write("="*60 + "\n")
    f.write("FEATURE DOCUMENTATION FOR NEPAL VULNERABILITY ANALYSIS\n")
    f.write("="*60 + "\n\n")
    
    f.write("1. RAW PERCENTAGE FEATURES\n")
    f.write("-"*40 + "\n")
    f.write(f"Total: {len(raw_pct_features)}\n\n")
    for feat in sorted(raw_pct_features):
        f.write(f"  • {feat}\n")
    
    f.write("\n\n2. ENGINEERED FEATURES\n")
    f.write("-"*40 + "\n")
    f.write(f"Total: {len(engineered_ml_features)}\n\n")
    for feat in engineered_ml_features:
        f.write(f"  • {feat}\n")
        # Add description
        if 'RATIO' in feat:
            f.write(f"    Ratio of good quality to poor quality\n")
        elif 'DIVERSITY' in feat:
            f.write(f"    Simpson's Diversity Index (higher = more diverse)\n")
    
    f.write("\n\n3. CATEGORICAL FEATURES\n")
    f.write("-"*40 + "\n")
    f.write(f"Total: {len(categorical_features)}\n\n")
    for feat in categorical_features:
        f.write(f"  • {feat}\n")
    
    f.write("\n\n4. TARGET VARIABLES (NOT FEATURES!)\n")
    f.write("-"*40 + "\n")
    f.write("  • TARGET_CATEGORY: Vulnerability category (Low, Medium, High, Very High)\n")
    f.write("  • TARGET_SCORE: Normalized vulnerability score (0-100)\n")
    
    f.write("\n\n5. EXCLUDED VARIABLES (to avoid data leakage)\n")
    f.write("-"*40 + "\n")
    for feat in vuln_indices:
        f.write(f"  • {feat} (used to create target, excluded from features)\n")

print(f"   ✓ Feature documentation: {FINAL_PATH / 'feature_documentation.txt'}")

# ============================================================================
# STEP 10: Create visualizations
# ============================================================================

print("\n10. Creating visualizations...")

try:
    # Set style
    plt.style.use('seaborn-v0_8-darkgrid')
    
    # 1. Score distribution
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Histogram
    axes[0].hist(df_engineered['VULNERABILITY_SCORE_NORM'], bins=15, 
                 edgecolor='black', alpha=0.7, color='skyblue', density=True)
    axes[0].set_title('District Vulnerability Score Distribution', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Vulnerability Score (0-100)', fontsize=12)
    axes[0].set_ylabel('Density', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Add mean and median lines
    mean_score = df_engineered['VULNERABILITY_SCORE_NORM'].mean()
    median_score = df_engineered['VULNERABILITY_SCORE_NORM'].median()
    axes[0].axvline(mean_score, color='red', linestyle='--', linewidth=2, label=f'Mean: {mean_score:.1f}')
    axes[0].axvline(median_score, color='green', linestyle='--', linewidth=2, label=f'Median: {median_score:.1f}')
    axes[0].legend()
    
    # Boxplot by category
    category_order = ['Low', 'Medium', 'High', 'Very High']
    box_data = [df_engineered[df_engineered['VULNERABILITY_CATEGORY'] == cat]['VULNERABILITY_SCORE_NORM'] 
                for cat in category_order]
    
    box = axes[1].boxplot(box_data, labels=category_order, patch_artist=True)
    colors = ['#2ecc71', '#f1c40f', '#e67e22', '#e74c3c']
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    axes[1].set_title('Vulnerability Score by Category', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Category', fontsize=12)
    axes[1].set_ylabel('Vulnerability Score', fontsize=12)
    axes[1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(VIS_PATH / 'vulnerability_distribution_boxplot.png', dpi=150, bbox_inches='tight')
    
    # 2. Category bar chart with province breakdown
    plt.figure(figsize=(12, 8))
    
    if 'PROVINCE' in df_engineered.columns:
        # Create cross-tabulation
        cross_tab = pd.crosstab(df_engineered['PROVINCE'], df_engineered['VULNERABILITY_CATEGORY'])
        cross_tab = cross_tab[category_order]  # Ensure order
        
        ax = cross_tab.plot(kind='bar', stacked=True, color=colors, 
                            edgecolor='black', linewidth=0.5, figsize=(12, 8))
        
        plt.title('Vulnerability Categories by Province', fontsize=16, fontweight='bold', pad=20)
        plt.xlabel('Province', fontsize=12)
        plt.ylabel('Number of Districts', fontsize=12)
        plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xticks(rotation=45, ha='right')
        
        # Add total counts on top of each bar
        for i, (province, row) in enumerate(cross_tab.iterrows()):
            total = row.sum()
            plt.text(i, total + 0.5, str(total), ha='center', va='bottom', fontsize=10)
        
        plt.tight_layout()
    else:
        # Simple bar chart if no province data
        category_counts = df_engineered['VULNERABILITY_CATEGORY'].value_counts().reindex(category_order)
        bars = plt.bar(category_counts.index, category_counts.values, 
                       color=colors, edgecolor='black', linewidth=1.5)
        
        plt.title('District Vulnerability Categories', fontsize=16, fontweight='bold')
        plt.xlabel('Category', fontsize=12)
        plt.ylabel('Number of Districts', fontsize=12)
        
        # Add labels
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, height + 0.5,
                    f'{int(height)}', ha='center', va='bottom', fontsize=11, fontweight='bold')
        
        plt.tight_layout()
    
    plt.savefig(VIS_PATH / 'vulnerability_categories_province.png', dpi=150, bbox_inches='tight')
    
    # 3. Correlation heatmap of engineered features
    plt.figure(figsize=(10, 8))
    
    # Select features for correlation
    corr_features = engineered_ml_features + ['VULNERABILITY_SCORE_NORM']
    corr_data = df_engineered[corr_features].corr()
    
    # Create mask for upper triangle
    mask = np.triu(np.ones_like(corr_data, dtype=bool))
    
    # Plot heatmap
    sns.heatmap(corr_data, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
                annot_kws={"size": 9})
    
    plt.title('Correlation: Engineered Features vs Vulnerability Score', 
              fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(VIS_PATH / 'engineered_features_correlation.png', dpi=150, bbox_inches='tight')
    
    # 4. Scatter plot: Vulnerability vs Key Features
    if engineered_ml_features:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        axes = axes.flatten()
        
        key_features = engineered_ml_features[:4]  # Plot first 4 engineered features
        
        for i, feature in enumerate(key_features):
            if i < len(axes):
                ax = axes[i]
                ax.scatter(df_engineered[feature], df_engineered['VULNERABILITY_SCORE_NORM'],
                          alpha=0.6, color='steelblue', edgecolor='black', linewidth=0.5)
                
                # Add trend line
                z = np.polyfit(df_engineered[feature], df_engineered['VULNERABILITY_SCORE_NORM'], 1)
                p = np.poly1d(z)
                ax.plot(df_engineered[feature], p(df_engineered[feature]), 
                       "r--", alpha=0.8, linewidth=2)
                
                # Calculate correlation
                corr = df_engineered[feature].corr(df_engineered['VULNERABILITY_SCORE_NORM'])
                ax.set_title(f'{feature}\nCorrelation: {corr:.3f}', fontsize=11)
                ax.set_xlabel(feature, fontsize=10)
                ax.set_ylabel('Vulnerability Score', fontsize=10)
                ax.grid(True, alpha=0.3)
        
        # Hide unused subplots
        for j in range(i + 1, len(axes)):
            axes[j].set_visible(False)
        
        plt.suptitle('Vulnerability Score vs Engineered Features', fontsize=16, fontweight='bold', y=1.02)
        plt.tight_layout()
        plt.savefig(VIS_PATH / 'vulnerability_vs_features.png', dpi=150, bbox_inches='tight')
    
    plt.close('all')
    print("   ✓ Created 4 visualization sets in results/figures/")
    
except Exception as e:
    print(f"   ⚠ Visualization error: {str(e)}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SUMMARY
# ============================================================================

print("\n" + "="*80)
print("FEATURE ENGINEERING COMPLETE - IMPROVED VERSION")
print("="*80)

print(f"\nSUMMARY:")
print(f"• Districts processed: {len(df_engineered)}")
print(f"• Raw percentage features: {len(raw_pct_features)}")
print(f"• Engineered ML features: {len(engineered_ml_features)}")
print(f"• Categorical features: {len(categorical_features)}")
print(f"• TOTAL ML features: {len(all_ml_features)}")
print(f"• Target variable created: {'VULNERABILITY_CATEGORY' in df_engineered.columns}")

print(f"\nDATA LEAKAGE PREVENTION:")
print(f"• Excluded from ML features: {len(vuln_indices)} vulnerability indices")
print(f"• Target created from: {', '.join(vuln_indices)}")
print(f"• ML features are independent of target construction")

print(f"\nDATASETS SAVED:")
print(f"1. nepal_districts_engineered.csv - All engineered features and targets")
print(f"2. nepal_districts_ml_ready.csv - Clean dataset for ML (features + target)")
print(f"3. feature_documentation.txt - Detailed feature descriptions")

print("\n" + "="*80)
print("NEXT STEPS FOR ML MODELING:")
print("="*80)
print("1. Data preprocessing:")
print("   • One-hot encode PROVINCE (if included)")
print("   • Scale numerical features")
print("   • Check for multicollinearity")
print("\n2. Classification modeling:")
print("   • Use TARGET_CATEGORY as y (categorical)")
print("   • Use all_ml_features as X")
print("   • Apply proper cross-validation")
print("\n3. Regression modeling (optional):")
print("   • Use TARGET_SCORE as y (continuous)")
print("   • Predict vulnerability scores")

print("\nYour data is now properly prepared for machine learning without data leakage!")

FEATURE ENGINEERING FOR VULNERABILITY ANALYSIS - IMPROVED VERSION

1. Loading data...
   ✓ Loaded 77 districts with 41 columns
   District columns:  ID DISTRICT_NAME
 14     Taplejung
 15 Sankhuwasabha
 16    Solukhumbu

2. Identifying feature groups...
   Walls: 8 features
     Samples: WAL_MUD_BONDED_BRICKS_STONE, WAL_CEMENT_BONDED_BRICKS_STONE, WAL_WOOD_PLANKS...
   Water: 9 features
     Samples: DRI_TAP_PIPED_WITHIN_PREMISES, DRI_TAP_PIPED_OUTSIDE_PREMISES, DRI_TUBEWELL_HANDPUMP...
   Toilets: 5 features
     Samples: TOI_FLUSH_PUBLIC_SEWERAGE, TOI_FLUSH_SEPTIC_TANK, TOI_PIT_TOILET...

3. Converting counts to percentages...
   ✓ Created 8 percentage features for Walls
   ✓ Created 9 percentage features for Water
   ✓ Created 5 percentage features for Toilets
   Total percentage features: 22

4. Creating vulnerability indices (for target only)...
   ✓ HOUSING_VULN: average of 5 features (TARGET ONLY)
   ✓ WATER_VULN: average of 4 features (TARGET ONLY)
   ✓ SANITATION_VULN: average

C:\Users\saurav\AppData\Local\Temp\ipykernel_1460\4224510854.py:418: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  box = axes[1].boxplot(box_data, labels=category_order, patch_artist=True)


   ✓ Created 4 visualization sets in results/figures/

FEATURE ENGINEERING COMPLETE - IMPROVED VERSION

SUMMARY:
• Districts processed: 77
• Raw percentage features: 22
• Engineered ML features: 5
• Categorical features: 0
• TOTAL ML features: 27
• Target variable created: True

DATA LEAKAGE PREVENTION:
• Excluded from ML features: 3 vulnerability indices
• Target created from: HOUSING_VULN, WATER_VULN, SANITATION_VULN
• ML features are independent of target construction

DATASETS SAVED:
1. nepal_districts_engineered.csv - All engineered features and targets
2. nepal_districts_ml_ready.csv - Clean dataset for ML (features + target)
3. feature_documentation.txt - Detailed feature descriptions

NEXT STEPS FOR ML MODELING:
1. Data preprocessing:
   • One-hot encode PROVINCE (if included)
   • Scale numerical features
   • Check for multicollinearity

2. Classification modeling:
   • Use TARGET_CATEGORY as y (categorical)
   • Use all_ml_features as X
   • Apply proper cross-validation

3.